# Carni Herbi Omni

The following is basically just copy&paste of the code of the animal example in Guttag chapter23 p395. Therefore there isn't much explanation. \
Note: It is necessarry to have the dentalFormulas.txt and GTag.py files saved in the same folder as this notebook, or to adjust the code accordingly. dentalFormulas.txt is the dataset Guttag uses for the example and GTag.py is a collection of classes and functions that Guttag introduces before starting with the actual animal example. 

In [62]:
import numpy as np
import pylab
import random
from GTag import *

We want to store the data in a textfile and thus need a function to read such a file. It will store the information in 3 lists as features, labels and species names.

In [83]:
def readMammalData(fName):
    dataFile = open(fName, 'r')
    numFeatures = 0
    #Process lines at top of file
    for line in dataFile: #Find number of features
        if line[0:6] == '#Label': #indicates end of features
            break
        if line[0:5] != '#Name':
             numFeatures += 1
    featureVals = []

    #Produce featureVals, speciesNames, and labelList
    featureVals, speciesNames, labelList = [], [], []
    for i in range(numFeatures):
        featureVals.append([])

    #Continue processing lines in file, starting after comments
    for line in dataFile:
        #remove newline, then split
        dataLine = line[:-1].split(',')
        speciesNames.append(dataLine[0])
        classLabel = dataLine[-1]
        labelList.append(classLabel)
        for i in range(numFeatures):
            featureVals[i].append(float(dataLine[i+1]))

    #Use featureVals to build list containing the feature vectors
    #for each mammal
    featureVectorList = []
    for mammal in range(len(speciesNames)):
        featureVector = []
        for feature in range(numFeatures):
            featureVector.append(featureVals[feature][mammal])
            featureVectorList.append(featureVector)
    return featureVectorList, labelList, speciesNames

Reading the data and checking if it actually worked.

In [113]:
features,labels,species = readMammalData("dentalFormulas.txt")

# check if it worked
print(features[0])
print(labels[0])
print(species[0])

[3.0, 1.0, 3.0, 1.0, 3.0, 1.0, 3.0, 2.0, 10.0]
1
Badger


The KMC algorithm as introduced doesn't work with 3 lists and we need to further process the data into an example structure.

In [114]:
def buildMammalExamples(featureList, labelList, speciesNames):
    examples = []
    for i in range(len(speciesNames)):
        features = pylab.array(featureList[i])
        example = Example(speciesNames[i], features, labelList[i])
        examples.append(example)
    return examples

Function call and test

In [115]:
# call
examples = buildMammalExamples(features, labels,species)

# test
print(examples[0].features)
print(examples[0].label)
print(examples[0].name)

[ 3.  1.  3.  1.  3.  1.  3.  2. 10.]
1
Badger


Now we merge this with the KMC algorithm and some text output in a single funtion to allow for easy playing around.

In [118]:
def testTeeth(numClusters, numTrials):
    features, labels, species = readMammalData('dentalFormulas.txt')
    examples = buildMammalExamples(features, labels, species)
    bestClustering = trykmeans(examples, numClusters, numTrials)
    for c in bestClustering:
        names = ''
        for p in c.members():
            names += p.getName() + ', '
        print('\n' + names[:-2]) #remove trailing comma and space
        herbivores, carnivores, omnivores = 0, 0, 0
        for p in c.members():
            if p.getLabel() == '0':
                herbivores += 1
            elif p.getLabel() == '1':
                carnivores += 1
            else:
                omnivores += 1
        print(herbivores, 'herbivores,', carnivores, 'carnivores,',omnivores, 'omnivores')

Since our goal is to cluster for the 3 kinds of eating habits (carni, herbi and omni), so we intuitively try numClusters=3. \
Note: Some seeds will lead to an error due to empty clusters. If that happens just try a different one

In [119]:
random.seed(3)
testTeeth(3,40)


Elk, Fox, Fur seal, Grey seal, Guinea pig, Human, Jaguar, Kangaroo, Lion
3 herbivores, 5 carnivores, 1 omnivores

Badger, Bear, Beaver, Brown bat, Cat, Cougar, Cow, Deer, Dog, Red bat, Sea lion, Skunk, Squirrel, Wolf, Woodchuck
3 herbivores, 8 carnivores, 4 omnivores

Mink, Mole, Moose, Mouse, Pig, Porcupine, Rabbit, Raccoon, Rat
3 herbivores, 2 carnivores, 4 omnivores


We clearly see that the clustering didn't work as intended. On first sight we have 3 Herbis in each cluster and the rest is spread over the clusters. \
Taking a closer look, we see that there is a similarity between some of the animals in a cluster. The animals are clustered into weight classes, because the weight feature has a bigger range than the actual dental formulas. \
-----------The End---------- \
Guttag offers some approaches to solve the problem above, the following code is given for the sake of completeness. For explanation, see p.399 ff. in the Guttag text.

In [69]:
def zScaleFeatures(vals):
    """Assumes vals is a sequence of floats"""
    result = pylab.array(vals)
    mean = sum(result)/len(result)
    result = result - mean
    return result/np.std(result)

In [70]:
def iScaleFeatures(vals):
    """Assumes vals is a sequence of floats"""
    minVal, maxVal = min(vals), max(vals)
    fit = pylab.polyfit([minVal, maxVal], [0, 1], 1)
    return pylab.polyval(fit, vals)

Note: The following functions are named differently than the ones in Guttag to not overwrite our previous ones. 

In [71]:
def readMammalData2(fName, scale):
    dataFile = open(fName, 'r')
    numFeatures = 0
    #Process lines at top of file
    for line in dataFile: #Find number of features
        if line[0:6] == '#Label': #indicates end of features
            break
        if line[0:5] != '#Name':
             numFeatures += 1
    featureVals = []

    #Produce featureVals, speciesNames, and labelList
    featureVals, speciesNames, labelList = [], [], []
    for i in range(numFeatures):
        featureVals.append([])

    #Continue processing lines in file, starting after comments
    for line in dataFile:
        #remove newline, then split
        dataLine = line[:-1].split(',')
        speciesNames.append(dataLine[0])
        classLabel = dataLine[-1]
        labelList.append(classLabel)
        for i in range(numFeatures):
            featureVals[i].append(float(dataLine[i+1]))

    #Use featureVals to build list containing the feature vectors
    #for each mammal, scaling features as indicated
    for i in range(numFeatures):
        featureVals[i] = scale(featureVals[i])
    featureVectorList = []
    for mammal in range(len(speciesNames)):
        featureVector = []
        for feature in range(numFeatures):
            featureVector.append(featureVals[feature][mammal])
        featureVectorList.append(featureVector)
    return featureVectorList, labelList, speciesNames

In [72]:
def testTeeth2(numClusters, numTrials, scale = lambda x: x):
    features, labels, species = readMammalData2('dentalFormulas.txt', scale)
    examples = buildMammalExamples(features, labels, species)
    bestClustering = trykmeans(examples, numClusters, numTrials)
    for c in bestClustering:
        names = ''
        for p in c.members():
            names += p.getName() + ', '
        print('\n' + names[:-2]) #remove trailing comma and space
        herbivores, carnivores, omnivores = 0, 0, 0
        for p in c.members():
            if p.getLabel() == '0':
                herbivores += 1
            elif p.getLabel() == '1':
                carnivores += 1
            else:
                omnivores += 1
        print(herbivores, 'herbivores,', carnivores, 'carnivores,',omnivores, 'omnivores')

In [61]:
random.seed(0) #so two clusterings starts with same seed
print('Clustering without scaling')
testTeeth2(3, 40)
random.seed(0) #so two clusterings starts with same seed
print('\n---------*---------*---------*---------*---------*---------')
print('\nClustering with z-scaling')
testTeeth2(3, 40, zScaleFeatures)
print('\n---------*---------*---------*---------*---------*---------')
print('\nClustering with i-scaling')
testTeeth2(3, 40, iScaleFeatures)

Clustering without scaling

Moose
1 herbivores, 0 carnivores, 0 omnivores

Bear, Cow, Deer, Elk, Fur seal, Grey seal, Lion, Sea lion
3 herbivores, 4 carnivores, 1 omnivores

Badger, Beaver, Brown bat, Cat, Cougar, Dog, Fox, Guinea pig, Human, Jaguar, Kangaroo, Mink, Mole, Mouse, Pig, Porcupine, Rabbit, Raccoon, Rat, Red bat, Skunk, Squirrel, Wolf, Woodchuck
5 herbivores, 11 carnivores, 8 omnivores

---------*---------*---------*---------*---------*---------

Clustering with z-scaling

Badger, Bear, Brown bat, Cat, Cougar, Dog, Fox, Fur seal, Grey seal, Human, Jaguar, Lion, Mink, Mole, Pig, Raccoon, Red bat, Sea lion, Skunk, Wolf
0 herbivores, 15 carnivores, 5 omnivores

Beaver, Guinea pig, Kangaroo, Mouse, Porcupine, Rabbit, Rat, Squirrel, Woodchuck
5 herbivores, 0 carnivores, 4 omnivores

Cow, Deer, Elk, Moose
4 herbivores, 0 carnivores, 0 omnivores

---------*---------*---------*---------*---------*---------

Clustering with i-scaling

Beaver, Guinea pig, Kangaroo, Mouse, Porcupine, 